In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import operator
import torch
import torchvision.datasets as da
import torchvision.transforms as transforms
import torchvision
import os
import cv2
from PIL import Image
from PIL import ImageFilter
from PIL import ImageEnhance
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
DIR_G_TRAIN = "drive/MyDrive/COMP9417/RiceLeafs_G/train/"
DIR_G_VALID = "drive/MyDrive/COMP9417/RiceLeafs_G/validation/"
classes1 = os.listdir(DIR_G_TRAIN)
classes2 = os.listdir(DIR_G_VALID)

In [ ]:
# get train and validation dataset
classes = {'BrownSpot':0,'LeafBlast':1,'Healthy':2,'Hispa':3}
x_train = []
y_train = []
x_val = []
y_val = []
for index1 in range(len(classes1)):
  leaf = os.listdir(DIR_G_TRAIN + classes1[index1])
  for index2 in range(len(leaf)):
    im = cv2.imread(DIR_G_TRAIN + classes1[index1] + '/' + leaf[index2])
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    gray = gray.reshape(1,-1)
    gray = gray[0]
    x_train.append(gray)
    y_train.append(classes[classes1[index1]])

for index1 in range(len(classes2)):
  leaf = os.listdir(DIR_G_VALID + classes2[index1])
  for index2 in range(len(leaf)):
    im = cv2.imread(DIR_G_VALID + classes2[index1] + '/' + leaf[index2])
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    gray = gray.reshape(1,-1)
    gray = gray[0]
    x_val.append(gray)
    y_val.append(classes[classes2[index1]])

In [ ]:
# centralized data to [-1,1], and mean = 0, standard variance = 1
def centralized(data):
  central_data = []
  for item in range(len(data)):
    image = data[item].astype(float)
    tensor_image = image/255
    mean_image = np.mean(tensor_image)
    std_image = tensor_image.std()
    central_image = (tensor_image-mean_image)/std_image
    central_data.append(central_image)
  return central_data

# convert array to list 
def array_to_list(data):
  list_data = []
  for item in range(len(data)):
    image = list(data[item])
    temp = []
    for i in range(len(image)):
      num = np.float(image[i])
      if np.isnan(num) == True:
        temp.append(np.float(0))
      else:
        temp.append(num)
    list_data.append(temp)
  return list_data

In [ ]:
x_train = centralized(x_train)
x_train = array_to_list(x_train)
x_val = centralized(x_val)
x_val = array_to_list(x_val)

<ipython-input-4-05af50b417e0>:9: RuntimeWarning: invalid value encountered in true_divide
  central_image = (tensor_image-mean_image)/std_image
<ipython-input-4-05af50b417e0>:20: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  num = np.float(image[i])
<ipython-input-4-05af50b417e0>:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  temp.app

In [ ]:
print(len(x_train),len(x_val))

2684 671


In [ ]:
# Re-partition the dataset into train and validation
num_shuffle = list(range(3355))
np.random.shuffle(num_shuffle)
xx = []
xx.extend(x_train)
xx.extend(x_val)
yy = []
yy.extend(y_train)
yy.extend(y_val)
X_train = []
Y_train = []
X_val = []
Y_val = []
print(num_shuffle)
for i in range(3355):
  if i < 2684:
    X_train.append(xx[num_shuffle[i]])
    Y_train.append(yy[num_shuffle[i]])
  else:
    X_val.append(xx[num_shuffle[i]])
    Y_val.append(yy[num_shuffle[i]])
print(len(X_train),len(X_val))

[3291, 578, 2216, 1861, 1596, 1975, 773, 873, 1965, 468, 2006, 1018, 3068, 914, 182, 683, 1026, 2306, 2267, 417, 1602, 1943, 781, 716, 2712, 1920, 1642, 287, 2535, 473, 311, 1144, 3177, 690, 528, 1203, 2207, 281, 3031, 924, 87, 2888, 742, 3085, 834, 21, 843, 150, 2354, 3024, 119, 1662, 2468, 469, 2222, 1403, 2419, 404, 3061, 1666, 2588, 2331, 2111, 3257, 1731, 2791, 1265, 1155, 2388, 12, 336, 1188, 1905, 2737, 2697, 2947, 3239, 454, 2639, 2626, 641, 1445, 1584, 538, 1109, 893, 2563, 2569, 103, 622, 1730, 1115, 1703, 1947, 2509, 2700, 1878, 2431, 250, 1284, 797, 816, 1607, 1020, 2801, 108, 169, 369, 2436, 302, 1718, 2134, 1789, 3251, 2784, 2751, 656, 2328, 646, 2175, 2116, 669, 1728, 524, 2686, 446, 2318, 1883, 3338, 3235, 1196, 2928, 632, 1414, 2206, 739, 2685, 247, 1322, 2543, 693, 2725, 2469, 2885, 2941, 1128, 282, 1344, 2870, 894, 3226, 2939, 224, 2234, 296, 2618, 2321, 1231, 2632, 1249, 673, 2068, 2016, 248, 850, 90, 3309, 899, 2058, 3120, 2699, 1160, 3110, 815, 2032, 2291, 477, 19

In [ ]:
# sklearn Knn (Euclidean distance)
from sklearn.neighbors import KNeighborsClassifier
for k in range(2,26):
  neigh = KNeighborsClassifier(n_neighbors=k)
  neigh.fit(X_train, Y_train)
  y_pred = []
  for i in range(len(X_val)):
    y_pred.append(neigh.predict([X_val[i]]))
  num_correct = 0
  for i in range(len(y_val)):
    if y_pred[i] == Y_val[i]:
      num_correct += 1
  print(num_correct/len(Y_val))

0.35469448584202684
0.3621460506706408
0.39344262295081966
0.3979135618479881
0.4083457526080477
0.4157973174366617
0.4277198211624441
0.44113263785394935
0.4232488822652757
0.4336810730253353
0.4396423248882265
0.45007451564828616
0.44709388971684055
0.45305514157973176
0.45454545454545453
0.46199701937406856
0.47093889716840537
0.47540983606557374
0.47242921013412814
0.47391952309985097
0.47391952309985097
0.47391952309985097
0.47540983606557374
0.4828614008941878


In [ ]:
# get the best performing of train validation division after testing
finalx_train = X_train
finalx_val = X_val
finaly_train =Y_train
finaly_val = Y_val

In [ ]:
# sklearn K (Euclidean distance)
neigh = KNeighborsClassifier(n_neighbors=25)
neigh.fit(finalx_train, finaly_train)
y_pred = []
for i in range(len(finalx_val)):
  y_pred.append(neigh.predict([finalx_val[i]]))

num_correct = 0
Tp0,Tp1,Tp2,Tp3 = 0,0,0,0
Fp0,Fp1,Fp2,Fp3 = 0,0,0,0
Fn0,Fn1,Fn2,Fn3 = 0,0,0,0
for i in range(len(y_val)):
  if y_pred[i] == Y_val[i]:
    num_correct += 1
    if y_pred[i] == 0:
      Tp0 += 1
    if y_pred[i] == 1:
      Tp1 += 1
    if y_pred[i] == 2:
      Tp2 += 1
    if y_pred[i] == 3:
      Tp3 += 1
  else:
    if y_pred[i] == 0:
      Fp0 += 1
    if y_pred[i] == 1:
      Fp1 += 1
    if y_pred[i] == 2:
      Fp2 += 1
    if y_pred[i] == 3:
      Fp3 += 1
    if Y_val[i] == 0:
      Fn0 += 1
    if Y_val[i] == 1:
      Fn1 += 1
    if Y_val[i] == 2:
      Fn2 += 1
    if Y_val[i] == 3:
      Fn3 += 1
print(f"Accuracy is {num_correct/len(Y_val)}")
precision = (Tp0/(Tp0+Fp0)+Tp1/(Tp1+Fp1)+Tp2/(Tp2+Fp2)+Tp3/(Tp3+Fp3))/4
recall = (Tp0/(Tp0+Fn0)+Tp1/(Tp1+Fn1)+Tp2/(Tp2+Fn2)+Tp3/(Tp3+Fn3))/4
print(f"Precision is {precision}")
print(f"Recall is {recall}")
print(f"F1 is {2*precision*recall/(precision+recall)}")
print(Tp0/(Tp0+Fp0),Tp1/(Tp1+Fp1),Tp2/(Tp2+Fp2),Tp3/(Tp3+Fp3))
print(Tp0/(Tp0+Fn0),Tp1/(Tp1+Fn1),Tp2/(Tp2+Fn2),Tp3/(Tp3+Fn3))

Accuracy is 0.4828614008941878
Precision is 0.41911280467105705
Recall is 0.3437688542123739
F1 is 0.3777202583645257
0.3409090909090909 0.43636363636363634 0.5145631067961165 0.38461538461538464
0.3488372093023256 0.1509433962264151 0.828125 0.04716981132075472


In [ ]:
# sklearn Knn (Manhattan distance)
neigh = KNeighborsClassifier(n_neighbors=25, metric="manhattan")
neigh.fit(finalx_train, finaly_train)
y_pred = []
for i in range(len(finalx_val)):
  y_pred.append(neigh.predict([finalx_val[i]]))

num_correct = 0
Tp0,Tp1,Tp2,Tp3 = 0,0,0,0
Fp0,Fp1,Fp2,Fp3 = 0,0,0,0
Fn0,Fn1,Fn2,Fn3 = 0,0,0,0
for i in range(len(y_val)):
  if y_pred[i] == Y_val[i]:
    num_correct += 1
    if y_pred[i] == 0:
      Tp0 += 1
    if y_pred[i] == 1:
      Tp1 += 1
    if y_pred[i] == 2:
      Tp2 += 1
    if y_pred[i] == 3:
      Tp3 += 1
  else:
    if y_pred[i] == 0:
      Fp0 += 1
    if y_pred[i] == 1:
      Fp1 += 1
    if y_pred[i] == 2:
      Fp2 += 1
    if y_pred[i] == 3:
      Fp3 += 1
    if Y_val[i] == 0:
      Fn0 += 1
    if Y_val[i] == 1:
      Fn1 += 1
    if Y_val[i] == 2:
      Fn2 += 1
    if Y_val[i] == 3:
      Fn3 += 1
print(f"Accuracy is {num_correct/len(Y_val)}")
precision = (Tp0/(Tp0+Fp0)+Tp1/(Tp1+Fp1)+Tp2/(Tp2+Fp2)+Tp3/(Tp3+Fp3))/4
recall = (Tp0/(Tp0+Fn0)+Tp1/(Tp1+Fn1)+Tp2/(Tp2+Fn2)+Tp3/(Tp3+Fn3))/4
print(f"Precision is {precision}")
print(f"Recall is {recall}")
print(f"F1 is {2*precision*recall/(precision+recall)}")
print(Tp0/(Tp0+Fp0),Tp1/(Tp1+Fp1),Tp2/(Tp2+Fp2),Tp3/(Tp3+Fp3))
print(Tp0/(Tp0+Fn0),Tp1/(Tp1+Fn1),Tp2/(Tp2+Fn2),Tp3/(Tp3+Fn3))

Accuracy is 0.46646795827123694
Precision is 0.36332357790366654
Recall is 0.3373057444785724
F1 is 0.34983157574599827
0.3333333333333333 0.37142857142857144 0.5132382892057027 0.23529411764705882
0.36046511627906974 0.16352201257861634 0.7875 0.03773584905660377
